In [2]:
# # Required Packages for the Project RUN ONLY ONCE
# %pip install matplotlib
# %pip install nilearn
# %pip install openpyxl
# %pip install Path
# %pip install seaborn
# %pip install nltools
# %pip install scipy
# %pip install scikit-image
# %pip install nibabel
%pip install bctpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import copy
# from skimage import filters
# import json
# import nibabel as nib
# from nilearn.plotting import plot_img

import pandas as pd
from nilearn import image
from nilearn.image import load_img, index_img
import numpy as np
import scipy.io
import networkx as nx
import matplotlib as mt
import scipy as sc
import seaborn as sns
import bct

In [12]:
class SubjectData:
    
    def __init__(self, datasetPath, componentsPath):
        self.componentFileName = "adni_aa__sub01_component_ica_s1_.nii"
        self.timeCourseFileName = "adni_aa__sub01_timecourses_ica_s1_.nii"
        self.FNCmatFile = "adni_aa__postprocess_results.mat"
        self.component_key = "fnc_corrs_all"
        self.graphMetricGlobalMeasues = [
                # Global Measures
                # "Global efficiency", 
                # "Characteristic path length", 
                # "Clustering coefficient",
            ]
        self.graphMetricComponentMeasures = [
                # Component Measures
                # "Degree",
                # "Closeness centrality",
                "Participation coefficient"
            ]
        
        self.subjectsData = self.readFileofCSV(datasetPath) 
        self.componentData = self.readFileofCSV(componentsPath)

        self.modifySubjectsPath()

        self.domainList = self.componentData["icn_domain"]
        self.indexList = self.componentData["icn_index"]

    def modifySubjectsPath(self):
        for i in range(2404):
            self.subjectsData.at[i,"fc_dir"] = self.subjectsData.iloc[i]["fc_dir"].replace("FC","GIGICA")

    def readFileofCSV(self,path):
        fileData = pd.read_csv(path)
        return fileData
    
    def getDatasetPaths(self,subjectName):
        ans = list()
        for i in range(2404):
            if self.subjectsData.at[i,"ResearchGroup"] == subjectName:
                ans.append(self.subjectsData.at[i,"fc_dir"])
        return ans

In [13]:
class VoxelCounts(SubjectData):
    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "
    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.voxelCountMap = dict()
        self.prepareVoxelCountMap()

    def prepareVoxelCountMap(self):
        " \
            iterate over the list of Subjects like AD, CN, ....     \
        "
        for subjectName in self.subjectsToCalculate:
            self.voxelCountMap[subjectName]=dict()
            self.voxelCountMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            self.voxelCountMap[subjectName]["indexes"] = dict()
            for i in self.indexList:
                self.voxelCountMap[subjectName]["indexes"][i]=list()

    def calculateVoxelCount(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.voxelCountMap[subjectName]["paths"]:
                spacialMapName = path + self.componentFileName
                spacialMap = load_img(spacialMapName)
                for index in self.voxelCountMap[subjectName]["indexes"]:
                    actualIndex=index-1
                    componentImg = index_img(spacialMap, actualIndex)
                    componentImgData = componentImg.get_fdata()
                    component_threshold = 3*np.std(componentImgData)
                    component_voxelCount = np.count_nonzero(componentImgData > component_threshold)
                    self.voxelCountMap[subjectName]["indexes"][index].append(component_voxelCount)

In [ ]:

AD_Threshold = VoxelCounts('ADNI_demos.txt', 'NM_icns_info.csv', ["AD", "CN"])
AD_Threshold.calculateVoxelCount()

In [64]:
class GraphMetrics(SubjectData):

    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "

    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.graphMetricMap = dict()
        self.prepareGraphMetricMap()

    def prepareGraphMetricMap(self):
        for subjectName in self.subjectsToCalculate:
            self.graphMetricMap[subjectName]=dict()
            self.graphMetricMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            for graphMetric in self.graphMetricGlobalMeasues:
                self.graphMetricMap[subjectName][graphMetric]=list()
            
            for graphMetric in self.graphMetricComponentMeasures:
                self.graphMetricMap[subjectName][graphMetric] = dict()
                for ind in self.indexList:
                    self.graphMetricMap[subjectName][graphMetric][ind]=list()

    def loadMatFile(self,filePath):
        componentDict = scipy.io.loadmat(filePath+self.FNCmatFile)
        return componentDict[self.component_key]

    def prepareFNCMatrix(self,componentData):
        selected_component = np.zeros((53,53), dtype=np.float64).reshape(53,53)
        for i in range(53):
            for j in range(i+1, 53):
                # if componentData[self.indexList[i]-1][self.indexList[j]-1] >=0 : 
                selected_component[i][j]=componentData[self.indexList[i]-1][self.indexList[j]-1]
        selected_component += selected_component.T

        # Finding correlation matrix
        corrs = pd.DataFrame(selected_component)
        correlation_matrix = corrs.corr()
        correlation_numpy_array = correlation_matrix.to_numpy(dtype=np.float64)

        # Finding Thresholded correlation matrix
        row,column = correlation_numpy_array.shape
        for i in range(row):
            for j in range(column):
                if i!=j and correlation_numpy_array[i][j]<0:
                    correlation_numpy_array[i][j]=0

        input_matrix = nx.from_numpy_array(correlation_numpy_array)
        return input_matrix

    def calculateGraphMetrics(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.graphMetricMap[subjectName]["paths"]:
                fncMatrix = self.prepareFNCMatrix(self.loadMatFile(path))

                # for global measures:
                for graphMetric in self.graphMetricGlobalMeasues:
                    if graphMetric == "Global efficiency":
                        globalEfficieny = nx.global_efficiency(fncMatrix)
                        self.graphMetricMap[subjectName][graphMetric].append(globalEfficieny)

                    elif graphMetric == "Characteristic path length":
                        characteristicPathLength = nx.average_shortest_path_length(fncMatrix, weight='weight')
                        self.graphMetricMap[subjectName][graphMetric].append(characteristicPathLength)

                    elif graphMetric == "Clustering coefficient":
                        clusterCofficient = nx.clustering(fncMatrix, weight='weight')
                        clusterCofficient_network = 0
                        for i in clusterCofficient:
                            clusterCofficient_network += clusterCofficient[i]
                        clusterCofficient_network /= 53
                        self.graphMetricMap[subjectName][graphMetric].append(clusterCofficient_network)

                # for component measures
                for graphMetric in self.graphMetricComponentMeasures:
                    if graphMetric == "Degree":
                        all_nodes_degree = nx.degree(fncMatrix, weight= 'weight')
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_degree[ind])
                    elif graphMetric == "Closeness centrality":
                        all_nodes_cc = nx.closeness_centrality(fncMatrix, distance='weight', wf_improved=False)
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_cc[ind])
                    elif graphMetric == "Participation coefficient":
                        fncMatrix_numpy = nx.to_numpy_array(fncMatrix)
                        modularity = nx.algorithms.community.greedy_modularity_communities(fncMatrix)
                        sam=[]
                        for i in range(len(modularity)):
                            for j in list(modularity[i]):
                                sam.append(j)
                        sam = np.array(sam)
                        all_nodes__pc = bct.centrality.participation_coef(fncMatrix_numpy, ci=sam, degree='undirected')
                        for ind in range(len(self.indexList)):
                            self.graphMetricMap[subjectName][graphMetric][self.indexList[ind]].append(all_nodes_cc[ind])
                        break
                break
            break

In [65]:

AD_GraphMetric = GraphMetrics('ADNI_demos.txt', 'NM_icns_info.csv', ["AD"])
AD_GraphMetric.calculateGraphMetrics()

[0, 36, 5, 6, 7, 46, 15, 48, 27, 28, 44, 41, 51, 52, 21, 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 42, 43, 45, 47, 49, 50]
[0.93656292 0.94143441 0.95154352 0.94907281 0.94899858 0.90801902
 0.90776505 0.9219665  0.95379509 0.95441073 0.95403784 0.95461473
 0.95788414 0.9591483  0.95630455 0.90610956 0.94894894 0.92552188
 0.93960691 0.92850273 0.94401441 0.92932434 0.94760942 0.93740168
 0.94867448 0.95971276 0.95637214 0.93449594 0.93171023 0.95085605
 0.95784541 0.94764673 0.95565297 0.951718   0.93302607 0.956303
 0.89481902 0.95262102 0.95925686 0.951548   0.95432362 0.92160262
 0.95068754 0.94568761 0.93358042 0.93615367 0.92097075 0.9605426
 0.93119554 0.95243221 0.95084784 0.94138517 0.93003145]


In [ ]:
GraphMetrics_Names = AD_GraphMetric.graphMetricNames
ans1 = list()

for subjectName in AD_GraphMetric.subjectsToCalculate:

    table = pd.DataFrame()
    for i in range(len(AD_Threshold.indexList)):
        table[AD_Threshold.domainList[i]+'('+str(AD_Threshold.indexList[i])+')'] = list()

    for i in GraphMetrics_Names:
        indexList = AD_Threshold.indexList
        corr_list = list()

        xlist = AD_GraphMetric.graphMetricMap[subjectName][i]
        xmean = np.mean(xlist)
        xval = xlist - xmean
        xsqu = np.sqrt(np.sum(np.square(xval)))

        for j in range(len(indexList)):

            ylist = AD_Threshold.voxelCountMap[subjectName]["indexes"][AD_Threshold.indexList[j]]
            ymean = np.mean(ylist)
            yval = ylist - ymean
            ysqu = np.sqrt(np.sum(np.square(yval)))

            num = 0
            for k in range(len(yval)):
                num += (xval[k]*yval[k])

            den = (xsqu * ysqu)
            ans = num/den
            corr_list.append(ans)

        table.loc[len(table.index)] = corr_list
    table.index = AD_GraphMetric.graphMetricNames
    ans1.append(table)

In [ ]:
temp = ans1[:]

In [ ]:
symvalue = 0.5
fileNames = ['Alzhiemeris', 'Controls']
for i in range(len(fileNames)):
        temp[i].to_csv(fileNames[i]+'.csv')
        s1 = temp[i].to_numpy()
        fig, ax = mt.pyplot.subplots(figsize=(54,15))
        columnsName = temp[i].columns.values
        rowsName = temp[i].index.values
        sd = sns.heatmap(
                        s1, cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0, vmin= -1*symvalue, vmax=symvalue,
                        xticklabels=columnsName, yticklabels=rowsName,  annot=True
                )
        sd.get_figure().savefig(fileNames[i]+str(symvalue)+'.png')

In [ ]:
# ans1[1].to_csv('./Controls.csv')

columns = temp[1].columns.values

rows = temp[1].index.values

# print(ans1[0][rows[0]][columns[0]] - ans1[1][rows[0]][columns[0]])

for i in rows:
    for j in columns:
        temp[1].at[i,j] -= temp[0].at[i,j]

# ans1[1].to_csv('./difference.csv')

st1 = temp[1].to_numpy()

fig, ax = mt.pyplot.subplots(figsize=(54,15))

sd = sns.heatmap (
                st1, 
                vmin= -1*symvalue, 
                vmax= symvalue,
                cmap='coolwarm', 
                square=True,
                ax=ax,
                linewidth='0.5',
                center=0, 
                cbar=True,
                xticklabels=columnsName, 
                yticklabels=rowsName,  
                annot=True
            )

sd.get_figure().savefig('difference'+str(symvalue)+'.png')

In [ ]:
"""
1 4d subject
    - 53 selected Components
        - each component voxel Count - index i

AD- 213
69  - [ 1_Sub_69Ind_voxelCount, 2_Sub_69Ind_VoxelCount,  ]
53  - [ ]

1 4D subject:
    - preprocess.mat file 
        - FNC matrix
            - Graph metric 
                - Glo
                - Charac
                - Coefficint

AD- 213
Global Efficieny - [ 1_Sub_mat_FNC_Global_Value, 2_Sub_mat_FNC_Global, ..... ]
Characterstic Path lenght - [ 1_sub, ........ ]


Correlation:
Global Efficiency Vs 69 [] - Correlation 0.123
Global Efficiency Vs 53 [] - Correlation -0.324
                                            0.372


"""